# Map Reduce

Eerst maken we een aparte directory aan voor alles wat we voor deze notebook gaan gebruiken in hdfs. Dit om conflicten of het overschrijven van gegevens te vermijden.

In [1]:
import pydoop.hdfs as hdfs

In [2]:
# make links with local and hdfs file systems
localFS = hdfs.hdfs(host='')
client = hdfs.hdfs(host='localhost', port=9000)

# if directory does not exists - make directory
if not client.exists('/user/bigdata/MapReduce'):
    client.create_directory('/user/bigdata/MapReduce')
client.set_working_directory('/user/bigdata/MapReduce')
print(client.working_directory())

# do some cleaning in case anything else than input.txt is present
for f in client.list_directory("."):
    if not f["name"].endswith("input.txt"):
        client.delete(f["name"], True) # True om aan te geven dat folders ook verwijderd moeten worden

2023-02-16 14:54:39,565 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/user/bigdata/MapReduce


## Wat is MapReduce

MapReduce is een programmeermodel om eenvoudig distributed data te verwerken.
Het is belangrijk om te realiseren dat de programma's die je hier schrijft een parallel uitgevoerd worden op verschillende stukjes data (De map-fase) om daarna in de reduce-fase tot een finale output teruggebracht te worden.
Er gebeuren 5 stappen bij het uitvoeren van een MapReduce programma
* Bepalen op welke nodes de code uitgevoerd wordt (wordt door YARN gedaan afhankelijk van de locatie van de blocks)
* Uitvoeren van de Map-code (Geschreven door de developer naar eigen wens)
* Shuffle, ouput van de map-fase doorsturen naar andere nodes die de resultaten gaan reduceren (Automatisch)
* uitvoeren van de Reduce-code (Geschreven door de developer naar eigen wens)
* Combineren van de reduce output tot 1 gehele/finale output (Automatisch)

Uit bovenstaand stappenplan is het duidelijk dat er twee zaken moeten geimplementeerd worden bij het schrijven van een MapReduce toepassing.
Echter zullen we eerst een aantal voorbeelden bestuderen met reeds bestaande implementaties om zo meer vertrouwd te geraken met de flow van MapReduce.

## Voorbeelden van bestaande applicaties

Reeds een aantal default MapReduce applications zijn mee geinstalleerd met Hadoop.
De jar die deze toepassingen bevat kan gevonden worden in hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.0.jar.
Wanneer je deze jar uitvoert met onderstaande commando krijg je een lijst met de beschikbare applicaties

In [3]:
!hadoop jar ~/hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.0.jar

An example program must be given as the first argument.
Valid program names are:
  aggregatewordcount: An Aggregate based map/reduce program that counts the words in the input files.
  aggregatewordhist: An Aggregate based map/reduce program that computes the histogram of the words in the input files.
  bbp: A map/reduce program that uses Bailey-Borwein-Plouffe to compute exact digits of Pi.
  dbcount: An example job that count the pageview counts from a database.
  distbbp: A map/reduce program that uses a BBP-type formula to compute exact bits of Pi.
  grep: A map/reduce program that counts the matches of a regex in the input.
  join: A job that effects a join over sorted, equally partitioned datasets
  multifilewc: A job that counts words from several files.
  pentomino: A map/reduce tile laying program to find solutions to pentomino problems.
  pi: A map/reduce program that estimates Pi using a quasi-Monte Carlo method.
  randomtextwriter: A map/reduce program that writes 10GB of r

In deze notebook gaan we vooral focussen op het typische probleem van wordcount.
Dit is een toepassing dat gaat tellen hoe vaak elk woord voorkomt in een bepaalde tekst.
Om meer informatie over deze toepassing te krijgen kan je gebruik maken van het volgende commando:

In [4]:
!hadoop jar ~/hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.0.jar wordcount

Usage: wordcount <in> [<in>...] <out>


Voor we deze applicatie kunnen uitvoeren moeten we eerst ervoor zorgen dat er input data beschikbaar is in HDFS.
Upload hiervoor de input.txt file met onderstaande code

In [5]:
localFS.copy("input.txt", client, "input.txt") # upload file to hdfs

0

Het toevoegen van dit bestand kan geverifieerd worden door het HDFS te gaan bekijken op volgende link [http://localhost:9870/explorer.html#/user/bigdata/05_MapReduce](http://localhost:9870/explorer.html#/user/bigdata/05_MapReduce)

Met onderstaande commando kan nu het precompiled word count example uitgevoerd worden

**LET OP:** Paden naar de files in de terminal worden steeds als in de terminal uitgevoerd met user bigdata. Hierdoor begint ons pad standaard op /user/bigdata/

In [6]:
!hadoop jar ~/hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.0.jar wordcount MapReduce/input.txt MapReduce/output

2023-02-16 14:59:49,178 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2023-02-16 14:59:50,952 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_1675953405945_0001
2023-02-16 14:59:52,004 INFO input.FileInputFormat: Total input files to process : 1
2023-02-16 14:59:52,841 INFO mapreduce.JobSubmitter: number of splits:1
2023-02-16 14:59:55,089 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1675953405945_0001
2023-02-16 14:59:55,090 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-02-16 14:59:56,418 INFO conf.Configuration: found resource resource-types.xml at file:/home/bigdata/hadoop/etc/hadoop/resource-types.xml
2023-02-16 14:59:56,506 INFO resource.ResourceUtils: Adding resource type - name = vram, units = G, type = COUNTABLE
2023-02-16 14:59:58,247 INFO impl.YarnClientImpl: Submitted application application_1675953405945_0001
2023-02-16 14:59:58,

Indien het bovenstaande commando een foutmelding geeft over het niet vinden van org.apache.hadoop.mapreduce.v2.app.MRAppMaster, voeg het volgende toe aan hadoop/etc/hadoop/mapred-site.xml

    <property>
	  	<name>yarn.app.mapreduce.am.env</name>
	  	<value>HADOOP_MAPRED_HOME=/home/bigdata/hadooop</value>
	</property>
	<property>
	  <name>mapreduce.map.env</name>
	  <value>HADOOP_MAPRED_HOME=/home/bigdata/hadooop</value>
	</property>
	<property>
	  <name>mapreduce.reduce.env</name>
	  <value>HADOOP_MAPRED_HOME=/home/bigdata/hadooop</value>
	</property>

Na dit toe te voegen, herstart yarn en hdfs.
Indien het hier nog niet mee opgelost is voer de volgende stappen uit:
* Voer "hadoop classpath" uit in de terminal
* Voeg het volgende toe aan hadoop/etc/hadoop/yarn-site.xml:

    <property>
        <name>yarn.application.classpath</name>
        <value>output van stap 1</value>
    </property>

Na het uitvoeren van het wordcount applicatie moet er een output.txt file aangemaakt zijn met de resultaten. **Bekijk dit nu.**
Het resultaat is een file met key-value pairs met de woorden als key en de frequentie/aantal keer dat de woorden voorkomen als key.

Probeer nu op basis van bovenstaande reeds bestaande applicaties de gemiddelde lengte van de woorden in de text te bepalen.

In [7]:
# uit te voeren op commando voor de gemiddelde lengte van de woorden te bekomen
!hadoop jar ~/hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.0.jar wordmean MapReduce/input.txt MapReduce/output_mean_length

2023-02-16 15:11:29,151 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2023-02-16 15:11:30,788 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_1675953405945_0002
2023-02-16 15:11:31,681 INFO input.FileInputFormat: Total input files to process : 1
2023-02-16 15:11:31,895 INFO mapreduce.JobSubmitter: number of splits:1
2023-02-16 15:11:32,472 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1675953405945_0002
2023-02-16 15:11:32,472 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-02-16 15:11:32,856 INFO conf.Configuration: found resource resource-types.xml at file:/home/bigdata/hadoop/etc/hadoop/resource-types.xml
2023-02-16 15:11:32,935 INFO resource.ResourceUtils: Adding resource type - name = vram, units = G, type = COUNTABLE
2023-02-16 15:11:33,121 INFO impl.YarnClientImpl: Submitted application application_1675953405945_0002
2023-02-16 15:11:33,

De resource manager houdt ook een overzicht bij van de uitgevoerde applicaties, hun status, runtime en eventuele loggings. Na bovenstaande commando's uit te voeren moet je iets gelijkaardigs zien als in de onderstaande screenshot.

![yarn of mapreduce](images/yarn_001.png)

## Zelf implementeren van MapReduce applicaties

Natuurlijk zijn er veel meer zaken mogelijk om te berekenen met map-reduce toepassingen dan de reeds gecompileerde in hadoop.
Zoals eerder aangehaald valt vooral het coderen van de Map- en Reducestap hierbij op de schouders van de developer.
De standaard programmeertaal van MapReduce is Java en dus ook het grootste deel van de documentatie over MapReduce is geschreven met behulp van Java.
Deze programmas moeten dan gecompileerd worden tot een jar dat geupload kan worden naar de overeenkomstige nodes en daar uitgevoerd.
De api overview van hadoop kan je [hier](https://hadoop.apache.org/docs/stable/api/org/apache/hadoop/mapred/package-summary.html) vinden.

De code voor het wordcount example ziet er als volgt uit:

In [14]:
%%file WordCount.java
// dit schrijft onderstaande cell naar een file in de huidige directory
import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class WordCount {
    
    // Mapper -> classes tussen <> zijn de classen van de (input key, input_value, output_key, output_value)
    public static class WCMapper extends Mapper<Object, Text, Text, IntWritable>{
        
        // hier komt de mapfunctie in
        public void map(Object key, Text value, Context context) throws IOException, InterruptedException {
            // map functie leest lijn per lijn
            // lijn splitsen in woorden
            // hello world
            StringTokenizer itr = new StringTokenizer(value.toString());
            // itr = [hello, world]
            while(itr.hasMoreTokens()){
                // hier zitten we woord per woord te lezen
                // stuur voor elk woord (woord, 1)
                Text word = new Text();
                word.set(itr.nextToken());
                System.out.println(word.toString());
                context.write(word, new IntWritable(1));
                // (hello, 1)
                // (word, 1)
            }
        }
    }
   
    // Reducer -> classes tussen <> zijn de classen van de (input key, input_value, output_key, output_value)
    public static class WCReducer extends Reducer<Text, IntWritable, Text, IntWritable>{
        
        public void reduce(Text key, Iterable<IntWritable> values, Context context) throws IOException, InterruptedException {
            // key = hello, values = [1, 1, 1, 1]
            int sum = 0;
            for (IntWritable val: values) {
                sum += val.get();
            }
            System.out.println(sum);
            context.write(key, new IntWritable(sum));
        }
    }
    
    // configure the MapReduce program
    public static void main(String[] args) throws Exception {
        Configuration conf = new Configuration();
        Job job = Job.getInstance(conf, "word count java");
        job.setJarByClass(WordCount.class);
        // configure mapper
        job.setMapperClass(WCMapper.class);
        // configure combiner (soort van reducer die draait op mapping node voor performantie)
        job.setCombinerClass(WCReducer.class);
        // configure reducer
        job.setReducerClass(WCReducer.class);
        // set output key-value classes
        job.setOutputKeyClass(Text.class);
        job.setOutputValueClass(IntWritable.class);
        // set input file (first argument passed to the program)
        FileInputFormat.addInputPath(job, new Path(args[0]));
        // set output file  (second argument passed to the program)
        FileOutputFormat.setOutputPath(job, new Path(args[1]));
        // In this case, we wait for completion to get the output/logs and not stop the program to early.
        System.exit(job.waitForCompletion(true) ? 0 : 1);
    }
}

Overwriting WordCount.java


Deze code bevat drie delen:
* De main() functie: verzorgt de configuratie van de uit te voeren taak. Geeft aan wat de Map en Reduce klassen zijn, wat de input is, hoe de output bewaard wordt ,...
* De Map-klasse met de map() functie bevat de code voor de mapping-fase
* De Reduce-klasse met de reduce() functie bevat de code voor de reduce-fase

De laatste twee klassen zijn hier gecodeerd als geneste klassen. Deze hadden ook in aparte files geplaatst kunnen worden.
Nu moet deze code eerst omgezet/gecompileerd worden tot een .jar file. Deze kan dan analoog uitgevoerd worden als hierboven met het voorbeeldcode.
Deze twee stappen kunnen uitgevoerd worden door onderstaande commando's.

In [15]:
# compileren tot .jar
!javac -d . WordCount.java
!jar cvf wordcounter.jar *.class
# execute
!hadoop jar wordcounter.jar WordCount MapReduce/input.txt MapReduce/output_java

added manifest
adding: WordCount$WCMapper.class(in = 1686) (out= 758)(deflated 55%)
adding: WordCount$WCReducer.class(in = 1768) (out= 767)(deflated 56%)
adding: WordCount.class(in = 1494) (out= 830)(deflated 44%)
2023-02-16 15:33:00,358 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2023-02-16 15:33:01,872 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.
2023-02-16 15:33:01,927 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_1675953405945_0003
2023-02-16 15:33:03,173 INFO input.FileInputFormat: Total input files to process : 1
2023-02-16 15:33:03,546 INFO mapreduce.JobSubmitter: number of splits:1
2023-02-16 15:33:04,198 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1675953405945_0003
2023-02-16 15:33:04,199 INFO mapreduce.JobSubmi

### Oefening

Kopieer nu bovenstaand java programma en pas het aan zodat het het aantal worden telt dat begint met elke letter van het alfabet (zet eerst de woorden om naar kleine letters).
Hiervoor moet je dus het resultaat van de itr.nextToken() eerst gaan omvormen om er enkel de eerste letter eruit te halen en deze om te zetten naar lowercase).
De nodige functies om dit resultaat kun je [hier](https://docs.oracle.com/javase/7/docs/api/java/lang/String.html) en [hier](https://docs.oracle.com/javase/8/docs/api/java/lang/Character.html) vinden.
Indien dit gelukt is zorg er ook voor dat je geen entries toevoegt indien het eerste character geen letter is.
Compileer dit programma naar een .jar en voer het uit.
Controleer het resultaat.

In [20]:
%%file StartLetter.java
import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class StartLetter {
    
    // Mapper -> classes tussen <> zijn de classen van de (input key, input_value, output_key, output_value)
    public static class WCMapper extends Mapper<Object, Text, Text, IntWritable>{
        
        // hier komt de mapfunctie in
        public void map(Object key, Text value, Context context) throws IOException, InterruptedException {
            // map functie leest lijn per lijn
            // lijn splitsen in woorden
            // hello world
            StringTokenizer itr = new StringTokenizer(value.toString());
            // itr = [hello, world]
            while(itr.hasMoreTokens()){
                // hello -> (h, 1)
                // worlds -> (w, 1)
                String woord = itr.nextToken();
                woord = woord.toLowerCase();
                char firstLetter = woord.charAt(0);
                if(Character.isLetter(firstLetter)){
                    Text word = new Text();
                    word.set(Character.toString(firstLetter));
                    context.write(word, new IntWritable(1));
                }
            }
        }
    }
   
    // Reducer -> classes tussen <> zijn de classen van de (input key, input_value, output_key, output_value)
    public static class WCReducer extends Reducer<Text, IntWritable, Text, IntWritable>{
        
        public void reduce(Text key, Iterable<IntWritable> values, Context context) throws IOException, InterruptedException {
            // key = hello, values = [1, 1, 1, 1]
            int sum = 0;
            for (IntWritable val: values) {
                sum += val.get();
            }
            System.out.println(sum);
            context.write(key, new IntWritable(sum));
        }
    }
    
    // configure the MapReduce program
    public static void main(String[] args) throws Exception {
        Configuration conf = new Configuration();
        Job job = Job.getInstance(conf, "start letter java");
        job.setJarByClass(WordCount.class);
        // configure mapper
        job.setMapperClass(WCMapper.class);
        // configure combiner (soort van reducer die draait op mapping node voor performantie)
        job.setCombinerClass(WCReducer.class);
        // configure reducer
        job.setReducerClass(WCReducer.class);
        // set output key-value classes
        job.setOutputKeyClass(Text.class);
        job.setOutputValueClass(IntWritable.class);
        // set input file (first argument passed to the program)
        FileInputFormat.addInputPath(job, new Path(args[0]));
        // set output file  (second argument passed to the program)
        FileOutputFormat.setOutputPath(job, new Path(args[1]));
        // In this case, we wait for completion to get the output/logs and not stop the program to early.
        System.exit(job.waitForCompletion(true) ? 0 : 1);
    }
}

Overwriting StartLetter.java


In [21]:
# compileren tot .jar
!javac -d . StartLetter.java
!jar cvf allJavaClasses.jar *.class # let op, dit gaat wordcount classes meenemen in de jar dus met het eerste argument kan je kiezen welke main gestart wordt
# execute voeg StartLetter toe -> start vanaf de main in deze klasse
!hadoop jar allJavaClasses.jar StartLetter MapReduce/input.txt MapReduce/output_java_exer

added manifest
adding: StartLetter$WCMapper.class(in = 1778) (out= 804)(deflated 54%)
adding: StartLetter$WCReducer.class(in = 1774) (out= 766)(deflated 56%)
adding: StartLetter.class(in = 1519) (out= 838)(deflated 44%)
adding: WordCount$WCMapper.class(in = 1686) (out= 758)(deflated 55%)
adding: WordCount$WCReducer.class(in = 1768) (out= 767)(deflated 56%)
adding: WordCount.class(in = 1494) (out= 830)(deflated 44%)
2023-02-16 15:48:27,746 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2023-02-16 15:48:28,481 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.
2023-02-16 15:48:28,575 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_1675953405945_0004
2023-02-16 15:48:29,202 INFO input.FileInputFormat: Total input files to process : 1
2023-02-16 15:48:

## Implementeren via Python

Hoewel het Hadoop Ecosysteem geprogrammeerd is in Java en het dus er goed mee samenwerkt, is het niet verplicht om Java te gebruiken om MapReduce applicaties te schrijven.
Java krijgt namelijk veel kritiek, vooral doordat er veel code nodig is om eenvoudige zaken te programmeren.
Om andere programmeertalen te gebruiken worden er verscheidene API's aangeboden door Hadoop, namelijk
* Hadoop Streaming
    * Communicatie via stdin/stdout
    * Gebruikt door hadoopy, mrjob, ...
* Hadoop Pipes
    * C++ interface voor Hadoop
    * Communicatie via sockets
    * Gebruikt door pydoop


### Mrjob

De eerste API die we bekijken is MrJobs.
Deze API maakt gebruik van de Hadoop Streaming API.
De voordelen van MrJobs zijn:
* Uitgebreidde documentatie
* Code kan lokaal uitgevoerd worden als test
* Data serialisatie gebeurt automatisch (nadeel van Streaming API)
* Werkt met Amazon Elastic MapReduce en Google Cloud Dataproc

Het grootste nadeel is dat de StreamingAPI niet de volledige kracht heeft van het Hadoop ecosysteem omdat alles omgezet wordt naar strings (jsons)

Installatie van deze package gebeurt als volgt:

In [ ]:
#!echo bigdata | sudo -S pip install mrjob

In [22]:
%%file wordcount_mrjob.py
from mrjob.job import MRJob

class MrWordCount(MRJob):
    def mapper(self, _, line):
        # 2 argumenten -> (key, value) , key is niet belangrijk dus _ , line is 1 lijn in het bestand
        # deze functie wordt dus lijn per lijn opgeroepen
        for word in line.split():
            # yield -> return die de functie niet stopt
            yield (word, 1)
    
    def reducer(self, word, counts):
        # counts is een soort list van de aantal die we yielden in de mapper
        yield (word, sum(counts))
        
if __name__ == '__main__':
    MrWordCount.run()

# let op: de output van deze file staat lokaal

Writing wordcount_mrjob.py


Deze code kan uitgevoerd worden door het commando hieronder.
Let op dat dit een lokale file gebruikt voor de output.
Indien je de output wil bewaren in het hdfs moet je dit nog uploaden.

In [23]:
!python wordcount_mrjob.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt > output.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/bigdata/hadoop/bin...
Found hadoop binary: /home/bigdata/hadoop/bin/hadoop
Using Hadoop version 3.3.0
Looking for Hadoop streaming jar in /home/bigdata/hadoop...
Found Hadoop streaming jar: /home/bigdata/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar
Creating temp directory /tmp/wordcount_mrjob.bigdata.20230216.151356.247324
uploading working dir files to hdfs:///user/bigdata/tmp/mrjob/wordcount_mrjob.bigdata.20230216.151356.247324/files/wd...
Copying other local files to hdfs:///user/bigdata/tmp/mrjob/wordcount_mrjob.bigdata.20230216.151356.247324/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar4927860455492347435/] [] /tmp/streamjob3343779745894689764.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/

### Pydoop

Het word-count example door gebruik te maken van pydoop ziet er uit als volgt:

In [26]:
%%file wordcount_script.py

# hier zet je beter code om alle directories weg te doen, anders krijg je vaak een fout dat je directory reeds bestaat

def mapper(_, text, context):
    for word in text.split():
        print(word)
        #raise Exception("oepsie")
        context.emit(word, 1)
        
def reducer(word, counts, context):
    context.emit(word, sum(counts))
    

Overwriting wordcount_script.py


Dit script kan uitgevoerd worden op de cluster als volgt:

**Let op:** Indien dit script heel lang blijft hangen op 0% is er waarschijnlijk iets misgegaan.
De timeout die gebruikt wordt om de applicatie af te sluiten is 10 minuten.
De snelste manier op de applicatie af te sluiten is door gebruik te maken van het commando:
    yarn application -kill <application-id>


In [28]:
!pydoop script wordcount_script.py MapReduce/input.txt MapReduce/output_python_script

2023-02-16 16:24:11,947 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-02-16 16:24:21,662 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2023-02-16 16:24:22,960 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_1675953405945_0007
2023-02-16 16:24:23,756 WARN mapreduce.JobResourceUploader: No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2023-02-16 16:24:23,987 INFO input.FileInputFormat: Total input files to process : 1
2023-02-16 16:24:24,327 INFO mapreduce.JobSubmitter: number of splits:1
2023-02-16 16:24:25,234 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1675953405945_0007
2023-02-16 16:24:25,235 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-02-16 16:24:25,753 INFO mapred.YARNRunner: Job jar is not present. Not adding a

Bovenstaande code werkt door gebruik te maken van een script, dit maakt het mogelijk om eenvoudige applicaties te schrijven.
Voor complexere zaken (vooral in het geval dat er een bepaalde state moet bijgehouden worden kunnen er ook Mapper en Reducer classes aangemaakt worden waarmee het mogelijk is om de volledige [Pydoop API](https://crs4.github.io/pydoop/tutorial/mapred_api.html#api-tutorial) te gebruiken.

In [31]:
%%file wordcount_pydoop.py
import pydoop.mapreduce.api as api
import pydoop.mapreduce.pipes as pipes

class Mapper(api.Mapper):
    def map(self, context):
        for w in context.value.split():
            context.emit(w, 1)
    
class Reducer(api.Reducer):
    def reduce(self, context):
        context.emit(context.key, sum(context.values))
        
FACTORY = pipes.Factory(Mapper, reducer_class=Reducer)

def main():
    pipes.run_task(FACTORY)

if __name__ == "__main__":
    main()

Overwriting wordcount_pydoop.py


In [33]:
!pydoop submit --upload-file-to-cache wordcount_pydoop.py wordcount_pydoop MapReduce/input.txt MapReduce/output_python2 --entry-point main

2023-02-16 16:39:26,832 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-02-16 16:39:33,119 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2023-02-16 16:39:33,911 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_1675953405945_0009
2023-02-16 16:39:34,340 WARN mapreduce.JobResourceUploader: No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2023-02-16 16:39:34,478 INFO input.FileInputFormat: Total input files to process : 1
2023-02-16 16:39:34,695 INFO mapreduce.JobSubmitter: number of splits:1
2023-02-16 16:39:35,239 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1675953405945_0009
2023-02-16 16:39:35,240 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-02-16 16:39:35,635 INFO mapred.YARNRunner: Job jar is not present. Not adding a

In [34]:
%%file wordcount_pydoop2.py
# met counters
import pydoop.mapreduce.api as api
import pydoop.mapreduce.pipes as pipes

class Mapper(api.Mapper):
    def __init__(self, context):
        super(Mapper, self).__init__(context)
        # constructor in python
        context.set_status("constructor mapper")
        self.input_words = context.get_counter("WORDCOUNT", "INPUT_WORDS")
        
    def map(self, context):
        words = context.value.split()
        for w in words:
            context.emit(w, 1)
        context.increment_counter(self.input_words, len(words))
    
class Reducer(api.Reducer):
    def __init__(self, context):
        super(Reducer, self).__init__(context)
        # constructor in python
        context.set_status("constructor reducer")
        self.output_words = context.get_counter("WORDCOUNT", "OUTPUT_WORDS")
        
    def reduce(self, context):
        context.emit(context.key, sum(context.values))
        context.increment_counter(self.output_words, 1)
        
FACTORY = pipes.Factory(Mapper, reducer_class=Reducer)

def main():
    pipes.run_task(FACTORY)

if __name__ == "__main__":
    main()

Writing wordcount_pydoop2.py


In [37]:
!pydoop submit --upload-file-to-cache wordcount_pydoop2.py wordcount_pydoop2 MapReduce/input.txt MapReduce/output_python4 --entry-point main

2023-02-16 16:50:58,967 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-02-16 16:51:08,574 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2023-02-16 16:51:09,434 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_1675953405945_0011
2023-02-16 16:51:09,887 WARN mapreduce.JobResourceUploader: No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2023-02-16 16:51:10,058 INFO input.FileInputFormat: Total input files to process : 1
2023-02-16 16:51:10,287 INFO mapreduce.JobSubmitter: number of splits:1
2023-02-16 16:51:10,829 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1675953405945_0011
2023-02-16 16:51:10,829 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-02-16 16:51:11,148 INFO mapred.YARNRunner: Job jar is not present. Not adding a

**Indien je een fout maakt in een mapreduce applicatie blijft deze hangen tot er een timeout is. Je kan deze afbreken door middel van het volgende commando:**

In [ ]:
!yarn application -kill {application_id} # see yarn

**Let in het commando hierboven ook op de entry point parameter. Deze is belangrijk omdat het anders niet gaat werken**

### Oefening

Pas nu ook bovenstaande python applicatie aan om het aantal woorden dat begint met elke letter te tellen. Let er ook hierbij op dat er geen verschil is tussen hoofd- en kleine letters.

Indien dit goed gelukt is, probeer ook de applicatie aan te passen om per letter de gemiddelde lengte van de woorden te berekenen.
Let er hierbij op dat het niet rechtstreeks mogelijk is om meerdere keren de volledige iterator met de values te gebruiken. 
Terwijl dit nodig is voor het gemiddelde te berekenen met behulp van de standaard methoden (len, sum, ...).
Een oplossing hiervoor is om gebruik te maken van de itertools package

    from itertools import tee
    
Meer informatie over het gebruik hiervan vind je [hier](https://www.geeksforgeeks.org/python-itertools-tee/).

Daarnaast is het ook belangrijk om te beseffen dat je meerdere keys kan emitten per woord indien gewenst.
Breid nu de applicatie uit om ook de maximum lengte van alle woorden te bepalen.
Denk hierbij erover na hoe je het onderscheid kan maken tussen de twee soorten keys.

In [49]:
%%file wordcount_pydoop_oefening.py
import pydoop.mapreduce.api as api
import pydoop.mapreduce.pipes as pipes
from itertools import tee

class Mapper(api.Mapper):
    def map(self, context):
        for w in context.value.split():
            if len(w) >0 and w[0].isalpha():
                context.emit(w[0].lower(), 1)
            context.emit("word length", len(w))
    
class Reducer(api.Reducer):
    def reduce(self, context):
        # let op dat je bij deze aanpak geen data van twee keys kan gebruiken
        if context.key == "word length":
            # error omdat len niet bestaat        
            #context.emit("gemiddelde", sum(context.values) / len(context.values))
            
            # met itertools (elementen niet bewaard in aparte rij)
            print('met itertools')
            it1, it2 = tee(context.values, 2)
            som = sum(it1)
            aantal = sum(1 for _ in it2)    # bereken de lengte omdat len() niet bestaat
            context.emit("gemiddelde lengte", som/aantal)
            
            # met list (minst big data manier)
            print('met list')
            l = list(context.values)
            context_emit("gemmiddelde lengte met list", sum(l) / len(l))
            
            print('met for') # manuele for loop
            som = 0
            aantal = 0
            for val in context.values:
                aantal += 1
                som += val
            context.emit("gemiddelde lengte met for", som/aantal)
        else:
            context.emit(context.key, sum(context.values))
        
FACTORY = pipes.Factory(Mapper, reducer_class=Reducer)

def main():
    pipes.run_task(FACTORY)

if __name__ == "__main__":
    main()

Overwriting wordcount_pydoop_oefening.py


In [50]:
!pydoop submit --upload-file-to-cache wordcount_pydoop_oefening.py wordcount_pydoop_oefening MapReduce/input.txt MapReduce/output_python_oef6 --entry-point main

2023-02-16 17:14:42,690 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-02-16 17:14:48,766 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at /0.0.0.0:8032
2023-02-16 17:14:49,504 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/bigdata/.staging/job_1675953405945_0016
2023-02-16 17:14:49,791 WARN mapreduce.JobResourceUploader: No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2023-02-16 17:14:49,910 INFO input.FileInputFormat: Total input files to process : 1
2023-02-16 17:14:50,092 INFO mapreduce.JobSubmitter: number of splits:1
2023-02-16 17:14:50,523 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1675953405945_0016
2023-02-16 17:14:50,524 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-02-16 17:14:50,780 INFO mapred.YARNRunner: Job jar is not present. Not adding a